In [1]:
import pandas
import re, json
import csv

import torch
import torch.nn as nn
from datasets import load_metric,Dataset,DatasetDict, load_dataset, Sequence, Value
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, BartForConditionalGeneration
from transformers import AutoTokenizer, Trainer

import evaluate

import numpy as np
import nltk
import os
import random
from sklearn.model_selection import train_test_split
from typing import List, Optional, Tuple, Union, Dict, Any

In [2]:
seed = 42
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
_numpy_rng = np.random.default_rng(seed)
random.seed(seed)
np.random.seed(seed)
torch.use_deterministic_algorithms(False)
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
max_input_length = 256
max_target_length = 128

In [5]:
model_checkpoint = "facebook/bart-large"
metric = evaluate.load("rouge")
model = BartForConditionalGeneration.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [6]:
dataset = load_dataset('samsum')

In [7]:
def tokenize_and_align_labels(examples):
    inputs = [doc for doc in examples['dialogue']]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors='pt', padding=True)

    with tokenizer.as_target_tokenizer():
        tokenized_inputs = tokenizer(examples["summary"], truncation=True, return_tensors='pt', padding=True)
        
    model_inputs['labels'] = tokenized_inputs['input_ids']
    return model_inputs

In [8]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

/home/pvakhari/miniconda3/envs/nlp_env/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3864: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [9]:
tokenized_datasets['train'] = tokenized_datasets['train'].remove_columns(['id','dialogue', 'summary'])
tokenized_datasets['validation'] = tokenized_datasets['validation'].remove_columns(['id','dialogue', 'summary'])
tokenized_datasets['test'] = tokenized_datasets['test'].remove_columns(['id','dialogue', 'summary'])

In [10]:
training_args = Seq2SeqTrainingArguments(
    output_dir="checkpoints/",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    save_total_limit=4,
    num_train_epochs=10,
    predict_with_generate=True,
    do_train=True,
    do_eval=True,
    fp16=True,
    logging_steps=1,
    save_strategy="epoch",
    greater_is_better=True,
    load_best_model_at_end=True,
    seed=42,
    generation_max_length=max_target_length,
)

In [11]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [12]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    print(f"Generated summary: {decoded_preds[0]}")

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    print(f"Gold summary: {decoded_labels[0]}")

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    # print(result)
    # result = {"rouge": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [13]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [14]:
trainer.train()

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.665100,0.536888,0.509400,0.269400,0.425700,0.425400,26.684600
2,0.917600,0.504365,0.517500,0.274800,0.425700,0.425500,28.160100
3,0.424600,0.492877,0.521400,0.278100,0.431800,0.431400,28.467000


Generated summary: A wants to get a puppy for her son. B and A will go to the animal shelter tomorrow afternoon. A took her son to the shelter last Monday and he liked it. A will get him one of those little dogs. 
Gold summary: A will go to the animal shelter tomorrow to get a puppy for her son. They already visited the shelter last Monday and the son chose the puppy. 
Generated summary: A wants to get a puppy for his son. B will take him to the animal shelter tomorrow afternoon. A took his son to the shelter last Monday and he liked it. 
Gold summary: A will go to the animal shelter tomorrow to get a puppy for her son. They already visited the shelter last Monday and the son chose the puppy. 
Generated summary: A wants to get a puppy for her son. B will take him to the animal shelter tomorrow afternoon.
Gold summary: A will go to the animal shelter tomorrow to get a puppy for her son. They already visited the shelter last Monday and the son chose the puppy. 


KeyboardInterrupt: 

In [1]:
!nvidia-smi

Wed Nov 15 16:16:49 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:A1:00.0 Off |                    0 |
| N/A   33C    P0              63W / 300W |  20016MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--